# CompBioAsia 2023
## Machine Learning Workshop - Exploring small molecule potency

by [@carlosmr12](https://twitter.com/carlosmr12)

## Outline
- Task Overview
- Data Exploring
- Supervised Learning
    - Regression (PPI inhibitory potency)
        - Decision Tree
        - Performance Evaluation
        - Random Forest - Ensemble methods
    - Classification (Less-potent vs Potent PPI inhibitors)

## Task Overview
The problem we will solve is **predicting inhibitory activity of small molecules against certain protein-protein interactions (PPIs)**.

Proteins rarely act alone as their functions tend to be regulated. Many molecular processes within a cell are dependents of PPIs. Some PPIs are involved in multiple aggregation-related diseases, such as Creutzfeldt–Jakob and Alzheimer's diseases. The discovery of novel molecules capable of inhibiting these processes can be of great importance for medicine.

Small molecules are low molecular weight molecules that include lipids, monosaccharides, second messengers, other natural products and metabolites, as well as drugs and other xenobiotics. They can interact with receptors and regulate biological processes.

The first thing we need is a data set with inhibitory activity values for real molecules. For this workshop, we will use datasets from [TIMBAL](https://pubmed.ncbi.nlm.nih.gov/23766369/) and [iPPIDB](https://pubmed.ncbi.nlm.nih.gov/33416858/).

The inhibitory activity values are reported in **-log(IC50)**. IC50 means how much of a particular inhibitory drug is needed to inhibit a given biological process or biological component by 50\%. It is measured in uM (micromolar).


### Install libraries

In [ ]:
# Downloading our list of requirements the github repository
!wget https://raw.githubusercontent.com/carlosmr12/compbioasia2023/master/requirements.txt

In [ ]:
# Installing all dependencies in the file we downloaded in the previous cell
!pip install -r requirements.txt --use-deprecated=legacy-resolver

### Loading general libraries

In [ ]:
# AUTO-RELOAD EXTENSIONS
%load_ext autoreload
%autoreload 2

# PLOTTING
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
# Set notebook mode to work in offline
pyo.init_notebook_mode()

# DATA MANIPULATION
import numpy as np
import pandas as pd

# PREVENTS GENERAL WARNINGS FROM SHOWING ON OUTPUT
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Data Exploration

In [ ]:
# Download our dataset from the github repository
!wget https://raw.githubusercontent.com/carlosmr12/compbioasia2023/master/data/ppi_inhibitors.csv

In [ ]:
df_data = pd.read_csv("ppi_inhibitors.csv") # Load data from file to a DataFrame structure
print(df_data.shape) # .shape displays how the dataframe (matrix) looks like
df_data.head() # .head() displays the first few items in the dataframe

Data distribution based on data sources

In [ ]:
print(df_data['database'].value_counts())

fig = px.pie(df_data, values=df_data['database'].value_counts().values,
             names=df_data['database'].value_counts().index,
             title='Data sources')

fig.show(renderer="colab")

The molecules of the dataset are represented through **SMILES**. SMILES is a chemical notation that allows representation of a chemical structure.  They can be represented using simple vocabulary (atom and bond symbols), and few grammar rules:

- ***2-Propanol would be “CC(O)C”***

- ***2-Methylbutanal would be “CC(C)CC(=O)”.***

Using a type of SMILES called ISOMERIC SMILES it is even possible to represent specific isotopism, configuration about double bonds, and chirality.

Below you can see some examples of SMILES in our data:


2D depiction of small molecules based on the SMILES format

In [ ]:
# IMPORTING LIBRARIES FROM RDKIT (Open source toolkit for cheminformatics)
from rdkit.Chem.Draw import IPythonConsole #RDKit drawing
# A few settings to improve the quality of structures
from rdkit.Chem import rdDepictor
IPythonConsole.ipython_useSVG = True
rdDepictor.SetPreferCoordGen(True)

from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import Chem

In [ ]:
print(df_data.iloc[0]['SMILES'])
m = Chem.MolFromSmiles(df_data.iloc[0]['SMILES'])
m

In [ ]:
print(df_data.iloc[-1]['SMILES'])
m = Chem.MolFromSmiles(df_data.iloc[-1]['SMILES'])
m

In [ ]:
print(df_data.iloc[1]['SMILES'])
m = Chem.MolFromSmiles(df_data.iloc[2]['SMILES'])
m

What does the distribution of our **target** variable look like?

In [ ]:
target = "Experimental IC50 (log10)"
fig = px.histogram(df_data, x=target)
fig.show(renderer="colab")

Why do we would preferably work with the target variable in a log scale?

In [ ]:
df_data['inverse_log'] = 10**df_data[target]
fig = px.histogram(df_data, x='inverse_log')
fig.show(renderer="colab")

It is also important to note that our datasets covers, besides a long range of concentrations, also a range of values for diferent properties. This is also important to generate more generalised  models.

An important concept here is **feature**. A feature is a property of the object you’re trying to predict. It can also be referred to as indepent variable, since it is a fixed property of the data point and it does not depend of others variables. These independent variables are essential, because the algorithms need characteristics of the data points as support for the learning process and predicting the labels (in our case inhibitory activity).

In [ ]:
columns = ["MolLogP", "Acceptor_Count", "Donor_Count", "NumRotatableBonds",
           "RingCount", "MolWt"]

fig = make_subplots(rows=3, cols=2, start_cell="bottom-left")
fig.add_trace(go.Histogram(x=df_data[columns[0]], name=columns[0]), row=1, col=1)
fig.add_trace(go.Histogram(x=df_data[columns[1]], name=columns[1]), row=1, col=2)
fig.add_trace(go.Histogram(x=df_data[columns[2]], name=columns[2]), row=2, col=1)
fig.add_trace(go.Histogram(x=df_data[columns[3]], name=columns[3]), row=2, col=2)
fig.add_trace(go.Histogram(x=df_data[columns[4]], name=columns[4]), row=3, col=1)
fig.add_trace(go.Histogram(x=df_data[columns[5]], name=columns[5]), row=3, col=2)

fig.update_yaxes(title="count")
fig.show(renderer="colab")

## Regression (PPI inhibitory potency)

### Data Preparation

The method **train_test_split()** used in the code below divides our data into two subsets:

- One subset to *train* the model
- A subset to evaluate or *test* how good your model is, which should
    - Be large enough to yield statistically meaningful results
    - Be representative of the data set as whole

*(Never train on test data)*

Splitting your dataset into training and test sets is very important and it is directly related to your models ability to *learn* patterns during the training step, which will ideally help to make it generalisable. Using the test set (a subset completely unseen during trainning) you can estimate the performance of the model when it is applied to new data points.

In [ ]:
from sklearn.model_selection import cross_val_predict, train_test_split

# What is the best TEST_SIZE?
TEST_SIZE = 0.2

columns = ["MolLogP", "Acceptor_Count", "Donor_Count", "NumRotatableBonds",
           "RingCount", "MolWt"]
X = df_data[columns]
target = "Experimental IC50 (log10)"
y = df_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE)

fig = make_subplots(rows=1, cols=2, start_cell="bottom-left")

fig.add_trace(go.Histogram(x=y_train, name='Training set'), row=1, col=1)
fig.add_trace(go.Histogram(x=y_test, name='Test set'), row=1, col=2)
fig.update_xaxes(title=target)
fig.update_yaxes(title="count")
fig.show(renderer="colab")

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(max_depth=3)
print(regressor.get_params())

A commonly used validation method for ML models is known as *k-fold* cross-validation (CV).

The workflow for this approach consists of randomly splitting a data set ($D$) into $k$ mutually exclusive subsets, known as folds, $D_{1} , D_{2}, ..., D_{k}$ of approximately equal size. Secondly, the learning algorithm is trained and tested $k$ times; each time $t \in 1, 2, ..., k$, it is trained on $D − D_{t}$ and tested on $D_{t}$ .

The overall performance of the predictive model is then calculated based on the average of performances for each fold.

<img src="https://github.com/carlosmr12/compbioasia2023/blob/68c466bd897a9ccd1f2e78106200d5a3668fc903/static/cross_validation.png?raw=true">

In [ ]:
y_pred_train = cross_val_predict(regressor, X_train, y_train, cv=5)
regressor.fit(X_train, y_train)

y_pred_test = regressor.predict(X_test)

In [ ]:
from scipy.stats import pearsonr, kendalltau, spearmanr
from sklearn.metrics import mean_squared_error
from math import sqrt
from tabulate import tabulate

def regression_metrics(y_train, y_pred_train, y_test, y_pred_test):
    rmse_train = round(sqrt(mean_squared_error(y_train, y_pred_train)), 2)
    pearsons_train = round(pearsonr(y_train, y_pred_train)[0], 2)
    kendalls_train = round(kendalltau(y_train, y_pred_train)[0], 2)
    spearmans_train = round(spearmanr(y_train, y_pred_train)[0], 2)
    rmse_test = round(sqrt(mean_squared_error(y_test, y_pred_test)), 2)
    pearsons_test = round(pearsonr(y_test, y_pred_test)[0], 2)
    kendalls_test = round(kendalltau(y_test, y_pred_test)[0], 2)
    spearmans_test = round(spearmanr(y_test, y_pred_test)[0], 2)

    d = [ ["RMSE", rmse_train, rmse_test],
         ["Pearson's", pearsons_train, pearsons_test],
         ["Kendall's", kendalls_train, kendalls_test],
         ["Spearman's", spearmans_train, spearmans_test]]

    print(tabulate(d, headers=["Metric", "Training", "Test"]))
    return

**Root Mean Squared Error**

Root Mean Squared Error (RMSE) is a metric that tells us the average distance between the predicted values from the model and the actual values in the dataset (error). The lower the RMSE, the better a given model is able to “fit” a given dataset.

$\text{RMSE}(y, \bar{y}) = \sqrt{\frac{\sum_{i=1}^{N} (y_i - \bar{y}_i)^2}{N}}$

**Pearson's Correlation Coefficient**

The Pearson correlation coefficient is probably the most widely used measure for **linear relationships** between two normal distributed variables and thus often just called "correlation coefficient". Generally, the Pearson coefficient is obtained via a **Least-Squares fit** and a value of ***1*** represents a ***perfect positive relation-ship***, ***-1*** a ***perfect negative relationship***, and ***0*** indicates the ***absence of a relationship*** between variables.

$\rho = \frac{\text{cov}(X,Y)}{\sigma_x \sigma_y}$

**Spearman's Correlation Coefficient**

Spearman's *rho* can be understood as a rank-based version of Pearson's correlation coefficient, which can be used for variables that are not ***normal-distributed*** and have a ***non-linear relationship***. Also, its use is ***not only restricted to continuous data***, but can also be used in analyses of ordinal attributes.

$\rho = 1- {\frac {6 \sum d_i^2}{n(n^2 - 1)}}$

**Kendalls's Correlation Coefficient**

Similar to the Pearson correlation coefficient, Kendall's *tau* measures the degree of **a monotone relationship between variables** ( the variables tend to move in the **same relative direction**, but **not necessarily at a constant rate**), and like Spearman's **rho**, it calculates the dependence between ranked variables, which makes is feasible for non-normal distributed data. Kendall tau can be calculated for continuous as well as ordinal data. Roughly speaking, Kendall's **tau** distinguishes itself from Spearman's **rho** by a "harsher" penalizations.

$\tau = \frac{c-d}{c+d} = \frac{S}{
	\left(
	\begin{matrix}
 	n \\
 	2
\end{matrix}
\right)}
= \frac{2S}{n(n-1)}$

where $c$ is the number of *concordant pairs* and $d$ represents t=the number of *discordant pairs*

In [ ]:
regression_metrics(y_train, y_pred_train, y_test, y_pred_test)

In [ ]:
import statsmodels.api as sm
def regression_plots(y_train, pred_train, y_test, pred_test, target):
    fig = make_subplots(rows=1, cols=2, start_cell="bottom-left")

    fig.add_trace(go.Scatter(x=y_train, y=y_pred_train, name='Training set',
                             mode="markers"), row=1, col=1)
    fig.add_trace(go.Scatter(x=y_test, y=y_pred_test, name='Test set',
                             mode="markers"), row=1, col=2)

    x = sm.add_constant(y_train)
    p = sm.OLS(y_pred_train, x).fit().params
    x = np.arange(y_train.min(), y_train.max())
    y = p.const + p[target] * x
    fig.add_trace(go.Scatter(x=x, y=y, name='', mode="lines",
                             line=dict(dash='dash', color="black"),
                             showlegend=False), row=1, col=1)

    x = sm.add_constant(y_test)
    p = sm.OLS(y_pred_test, x).fit().params
    x = np.arange(y_test.min(), y_test.max())
    y = p.const + p[target] * x
    fig.add_trace(go.Scatter(x=x, y=y, name='', mode="lines",
                             line=dict(dash='dash', color="black"),
                             showlegend=False), row=1, col=2)

    fig.update_xaxes(title="Actual")
    fig.update_yaxes(title="Prediction")
    fig.show(renderer="colab")

In [ ]:
regression_plots(y_train, y_pred_train, y_test, y_pred_test, target)

Which feature is more important for the model?

In [ ]:
fig = px.bar(x=regressor.feature_names_in_, y=regressor.feature_importances_, title="Feature importance")
fig.update_layout(yaxis_title="Importance score", xaxis_title="Features")
fig.show(renderer="colab")

In [ ]:
from sklearn import tree

fig, axes = plt.subplots(nrows=1 ,ncols=1, figsize=(9,6), dpi=300) # AVOID RUNNING THIS ON DECISION TREES WITH max_depth < 3 . IT COULD TAKE SOME TIME TO FINISH
tree.plot_tree(regressor, feature_names=X_train.columns.tolist(), filled=True)
plt.show()

Impurity for regression in decision trees is generaly calculated in terms of "variance reduction".

The decision tree algorithm tries the minimise the inpurity of the nodes in the tree.

$ min\frac{1}{N}\sum_{i=1}^{i=N}(yi-\bar{y})^2$

Notice how the splits on the tree above try to "group" similar values (colours).

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor()
print(regressor.get_params())

In [ ]:
y_pred_train = cross_val_predict(regressor, X_train, y_train, cv=5)

regressor.fit(X_train, y_train)
y_pred_test = regressor.predict(X_test)

In [ ]:
regression_metrics(y_train, y_pred_train, y_test, y_pred_test)

In [ ]:
regression_plots(y_train, y_pred_train, y_test, y_pred_test, target)

Which features were more important to the model?

In [ ]:
fig = px.bar(x=regressor.feature_names_in_, y=regressor.feature_importances_, title="Feature importance")
fig.update_layout(yaxis_title="Importance score", xaxis_title="Features")
fig.show(renderer="colab")

### Exploring descriptors in the [RDKit](https://rdkit.org/docs/index.html) library

Try and improve the performance on the predictive models using descriptors from the list below

In [ ]:
ds_list = ["BalabanJ","BertzCT","Chi0","Chi0n","Chi0v","Chi1","Chi1n",
           # "Chi1v","Chi2n","Chi2v","Chi3n","Chi3v","Chi4n","Chi4v",
           # "HallKierAlpha","Kappa1","Kappa2","Kappa3","NHOHCount",
           # "NOCount","PEOE_VSA1","PEOE_VSA10","PEOE_VSA11","PEOE_VSA12",
           "PEOE_VSA13","PEOE_VSA14","PEOE_VSA2","PEOE_VSA3","PEOE_VSA4",
           "PEOE_VSA5","PEOE_VSA6","PEOE_VSA7","PEOE_VSA8","PEOE_VSA9",
           # "SMR_VSA1","SMR_VSA10","SMR_VSA2","SMR_VSA3","SMR_VSA4","SMR_VSA5",
           # "SMR_VSA6","SMR_VSA7","SMR_VSA8","SMR_VSA9","SlogP_VSA1","SlogP_VSA10",
           # "SlogP_VSA11","SlogP_VSA12","SlogP_VSA2","SlogP_VSA3","SlogP_VSA4",
           # "SlogP_VSA5","SlogP_VSA6","SlogP_VSA7","SlogP_VSA8","SlogP_VSA9",
           # "VSA_EState1","VSA_EState10","VSA_EState2","VSA_EState3",
           # "VSA_EState4","VSA_EState5","VSA_EState6","VSA_EState7",
           # "VSA_EState8","VSA_EState9","fr_Al_COO","fr_Al_OH","fr_Al_OH_noTert",
           # "fr_ArN","fr_Ar_COO","fr_Ar_N","fr_Ar_NH","fr_Ar_OH","fr_COO","fr_COO2",
           # "fr_C_O","fr_C_O_noCOO","fr_C_S","fr_HOCCN","fr_Imine","fr_NH0",
           # "fr_NH1","fr_NH2","fr_N_O","fr_Ndealkylation1","fr_Ndealkylation2",
           # "fr_Nhpyrrole","fr_SH","fr_aldehyde","fr_alkyl_carbamate",
           # "fr_alkyl_halide","fr_allylic_oxid","fr_amide","fr_amidine","fr_aniline",
           # "fr_aryl_methyl","fr_azide","fr_azo","fr_barbitur","fr_benzene",
           # "fr_benzodiazepine","fr_bicyclic","fr_diazo","fr_dihydropyridine",
           # "fr_epoxide","fr_ester","fr_ether","fr_furan","fr_guanido","fr_halogen",
           # "fr_hdrzine","fr_hdrzone","fr_imidazole","fr_imide","fr_isocyan",
           # "fr_isothiocyan","fr_ketone","fr_ketone_Topliss","fr_lactam","fr_lactone",
           # "fr_methoxy","fr_morpholine","fr_nitrile","fr_nitro","fr_nitro_arom",
           # "fr_nitro_arom_nonortho","fr_nitroso","fr_oxazole","fr_oxime",
           # "fr_para_hydroxylation","fr_phenol","fr_phenol_noOrthoHbond","fr_phos_acid",
           # "fr_phos_ester","fr_piperdine","fr_piperzine","fr_priamide",
           # "fr_prisulfonamd","fr_pyridine","fr_quatN","fr_sulfide","fr_sulfonamd",
           # "fr_sulfone","fr_term_acetylene","fr_tetrazole","fr_thiazole","fr_thiocyan",
           "fr_thiophene","fr_unbrch_alkane","fr_urea"]

Run the cell below to add your selected columns to every small molecule present in our database which is stored as a dataframe in the variable `df_data`

In [ ]:
from rdkit.Chem import Descriptors

def get_descriptor(row):
    molecule = Chem.MolFromSmiles(row['SMILES'])
    return getattr(Descriptors, ds)(molecule)

# For each item in the list of new descriptors you are adding
# we will use the .apply() method will execute the function
# "get_descriptor" (above) for every single entry in our
# dataframe

for ds in ds_list:
    df_data[ds] = df_data.apply(get_descriptor, axis=1)

In [ ]:
df_data.head()

In [ ]:
# What is the best TEST_SIZE?
TEST_SIZE = 0.2

columns = ["MolLogP", "Acceptor_Count", "Donor_Count", "NumRotatableBonds",
           "RingCount", "MolWt", "PEOE_VSA13", "PEOE_VSA14", "PEOE_VSA2",
           "PEOE_VSA3", "PEOE_VSA4", "PEOE_VSA5", "PEOE_VSA6", "PEOE_VSA7",
           "PEOE_VSA8", "PEOE_VSA9", "BalabanJ", "BertzCT", "Chi0",
           "Chi0n", "Chi0v", "Chi1", "Chi1n", "fr_thiophene",
           "fr_unbrch_alkane", "fr_urea"]

target = "Experimental IC50 (log10)"

X = df_data[columns]
y = df_data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE)

In [ ]:
regressor = RandomForestRegressor()

In [ ]:
y_pred_train = cross_val_predict(regressor, X_train, y_train, cv=5)

regressor.fit(X_train, y_train)
y_pred_test = regressor.predict(X_test)

In [ ]:
regression_metrics(y_train, y_pred_train, y_test, y_pred_test)

In [ ]:
regression_plots(y_train, y_pred_train, y_test, y_pred_test, target)

## Classification (Less-potent vs Potent PPI inhibitors)

In order to characterize the molecules of the dataset as potent or less potent we need to set a threshold value.

Finding this value is not a simple task, it depends of the biological question. According to Arkin et al, 2014 values > 5 are considered favorable for in vivo activity.

But you could select a threshold higher or lower in order to predict as positive only very potent small molecules or, vice-verse, respectively.

Arkin MR, Tang Y, Wells JA. Small-molecule inhibitors of protein-protein interactions: progressing toward the reality. Chem Biol. 2014 Sep 18;21(9):1102-14. DOI: [10.1016/j.chembiol.2014.09.001](10.1016/j.chembiol.2014.09.001). PMID: [25237857](https://pubmed.ncbi.nlm.nih.gov/25237857/)

Use the variable below to split the dataset into potent and less potent inhibitors

In [ ]:
THRESHOLD_POTENCY = 6

In [ ]:
df_data['potency_class'] = 'less_potent'
df_data.loc[df_data[target] > THRESHOLD_POTENCY, 'potency_class'] = 'potent'

fig = px.pie(df_data, values=df_data['potency_class'].value_counts().values, \
             names=df_data['potency_class'].value_counts().index, \
             color=df_data['potency_class'].value_counts().index, \
             title='PPI inhibitors potency', \
             color_discrete_map={'less_potent':'blue', 'potent':'red'})
fig.show(renderer="colab")

In [ ]:
target = 'potency_class'

# Basic features dataset
features = ["MolLogP", "Acceptor_Count", "Donor_Count", "NumRotatableBonds",
            "RingCount", "MolWt"]

X = df_data[features]
y = df_data[target]

# What is the best TEST_SIZE?
TEST_SIZE = 0.20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE,
                                                    random_state=42)

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(go.Pie(values=y_train.value_counts().values,
                     labels=y_train.value_counts().index,
                     title='Training set'), row=1, col=1)

fig.add_trace(go.Pie(values=y_test.value_counts().values,
                     labels=y_test.value_counts().index,
                     title='Test set'), row=1, col=2)

fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['red','blue']))

fig.show(renderer="colab")

#### Decision Tree

Decision trees are composed of if/else questions disposed in a hierarchical manner, following these questions the model is capable of reaching a decision. In the case of our question, the actual output are 'less_potent' or 'potent' labels. The decision to reach prediction is based on the features we used as input for the ML algorithm.

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=3)
y_pred_train = cross_val_predict(clf, X_train, y_train, cv=5)

clf.fit(X_train, y_train)
y_pred_test = clf.predict(X_test)

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def gen_train_test_cm(y_train, pred_train, y_test, pred_test, classes):
    print("Performance on Training (k-fold cross validation)")
    print(classification_report(y_train, pred_train))
    print("Performance on Test set")
    print(classification_report(y_test, pred_test))

    cm_train = confusion_matrix(y_train, pred_train)
    disp_train = ConfusionMatrixDisplay(confusion_matrix=cm_train)

    cm_test = confusion_matrix(y_test, pred_test)
    disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 9))

    disp_train.plot(ax=ax1, colorbar=False)
    disp_train.ax_.set_title("k-fold Cross-validation")
    disp_test.plot(ax=ax2, colorbar=False)
    disp_test.ax_.set_title("Test")
    plt.show()

**Precision** denotes the proportion of **Predicted Positive** cases that are **Actual Positives**.

$Precision = \frac{TP}{TP+FP}$

**Recall** measures the proportion of **Actual Positives** cases that are correctly **Predicted as Positives**.

$Recall = \frac{TP}{TP+FN}$

A combination of *Precision* and *Recall* in a harmonic mean denotes another metric known as **F-measure** or **F-score**.

$F score = 2 \times \frac{precision \times recall}{precision + recall}$

Precision, Recall and F-score values range from 0 to +1, the latter representing a perfect prediction for the class being evaluated.

In [ ]:
gen_train_test_cm(y_train, y_pred_train, y_test, y_pred_test, ['less_potent', 'potent'])

In [ ]:
def gen_train_test_roc(y_train, pred_train, y_test, pred_test, classes):
    y_train_tmp = label_binarize(y_train, classes=classes)
    y_pred_train_tmp = label_binarize(pred_train, classes=classes)

    y_test_tmp = label_binarize(y_test, classes=classes)
    y_pred_test_tmp = label_binarize(pred_test, classes=classes)

    fig = make_subplots(rows=1, cols=2, start_cell="bottom-left")

    fpr, tpr, thresholds = roc_curve(y_train_tmp, y_pred_train_tmp)
    score_train = round(auc(fpr, tpr), 2)
    fig.add_trace(go.Scatter(x=fpr, y=tpr, name='Training set - AUC:{}'.format(score_train),
                             stackgroup = 'one'), row=1, col=1)
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], name='', mode='lines',
                             line=dict(dash='dash', color="gray"),
                             showlegend=False), row=1, col=1)

    fpr, tpr, thresholds = roc_curve(y_test_tmp, y_pred_test_tmp)
    score_test = round(auc(fpr, tpr), 2)
    fig.add_trace(go.Scatter(x=fpr, y=tpr, name='Test set - AUC:{}'.format(score_test),
                             stackgroup = 'one'), row=1, col=2)
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], name='', mode='lines',
                             line=dict(dash='dash', color="gray"),
                             showlegend=False), row=1, col=2)

    fig.show(renderer="colab")

A less biased metric for classification tasks is the Area Under the ROC Curve (**AUC**), which considers the *True Positive Rate (TPR)*, which is equivalent to *sensitivity* and corresponds to the proportion of positive data points correctly classified; and also the *False Positive Rate (FPR)*, which is equal to $1-specificity$ that corresponds to the proportion of negative data that are wrongly considered as positive, regarding all negative data points. A Receiver Operating Curve (ROC) is then plotted using TPR versus FPR and the AUC is the area under such curve.

$TPR = \frac{TP}{TP+FN}$

$FPR = 1 - (\frac{TN}{TN+FP})$

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/13/Roc_curve.svg/800px-Roc_curve.svg.png">

In [ ]:
gen_train_test_roc(y_train, y_pred_train, y_test, y_pred_test, ['less_potent', 'potent'])

In [ ]:
fig, axes = plt.subplots(nrows=1 ,ncols=1, figsize=(9,6), dpi=300)
tree.plot_tree(clf, feature_names=X_train.columns.tolist(), class_names=['less_potent', 'potent'], filled=True)
plt.show()

Which features are more important for the model?

In [ ]:
fig = px.bar(x=clf.feature_names_in_, y=clf.feature_importances_, title="Feature importance")
fig.update_layout(yaxis_title="Importance score", xaxis_title="Features")
fig.show(renderer="colab")

Can we improve this with some of the RDKit descriptors we generated before?

In [ ]:
target = 'potency_class'

columns = ["MolLogP", "Acceptor_Count", "Donor_Count", "NumRotatableBonds",
           "RingCount", "MolWt", "PEOE_VSA13", "PEOE_VSA14", "PEOE_VSA2",
           "PEOE_VSA3", "PEOE_VSA4", "PEOE_VSA5", "PEOE_VSA6", "PEOE_VSA7",
           "PEOE_VSA8", "PEOE_VSA9", "BalabanJ", "BertzCT", "Chi0",
           "Chi0n", "Chi0v", "Chi1", "Chi1n", "fr_thiophene",
           "fr_unbrch_alkane", "fr_urea"]

X = df_data[features]
y = df_data[target]

# What is the best TEST_SIZE?
TEST_SIZE = 0.20

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE,
                                                    random_state=42)

In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=3)

In [ ]:
y_pred_train_fpr = cross_val_predict(clf, X_train, y_train, cv=5)

clf.fit(X_train, y_train)
y_pred_test_fpr = clf.predict(X_test)

In [ ]:
gen_train_test_cm(y_train, y_pred_train, y_test, y_pred_test, ['less_potent', 'potent'])

In [ ]:
gen_train_test_roc(y_train, y_pred_train, y_test, y_pred_test, ['less_potent', 'potent'])

What does the decision tree looks like?

In [ ]:
fig, axes = plt.subplots(nrows=1 ,ncols=1, figsize=(9,6), dpi=300)
tree.plot_tree(clf, feature_names=X_train.columns.tolist(), class_names=['less_potent', 'potent'], filled=True)
plt.show()

Which features are more important for the model?

In [ ]:
fig = px.bar(x=clf.feature_names_in_, y=clf.feature_importances_, title="Feature importance")
fig.update_layout(yaxis_title="Importance score", xaxis_title="Features")
fig.show(renderer="colab")

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
print(clf.get_params())

In [ ]:
y_pred_train = cross_val_predict(clf, X_train, y_train, cv=5)

clf.fit(X_train, y_train)
y_pred_test = clf.predict(X_test)

In [ ]:
gen_train_test_cm(y_train, y_pred_train, y_test, y_pred_test, ['less_potent', 'potent'])

In [ ]:
gen_train_test_roc(y_train, y_pred_train, y_test, y_pred_test, ['less_potent', 'potent'])

Which features are more important for the model?

In [ ]:
fig = px.bar(x=clf.feature_names_in_, y=clf.feature_importances_, title="Feature importance")
fig.update_layout(yaxis_title="Importance score", xaxis_title="Features")
fig.show(renderer="colab")